In [1]:
import numpy as np
import pandas as pd
import requests
import json

In [2]:
# load api keys from api_keys.txt file
mbta_api = 'https://api-v3.mbta.com/'

api_keys = dict()
with open('api_keys.txt', 'r') as f:
    for line in f.readlines():
        name, key = line.split('=')
        api_keys[name] = key.rstrip()

In [3]:
# setup google maps
import gmaps
import gmaps.datasets
gmaps.configure(api_key=api_keys['gmaps_api_key'])

In [4]:
r = requests.get(mbta_api+'routes/')
r.status_code

200

In [5]:
r.headers['content-type']
route_data = r.json()['data']

In [6]:
route_1 = [d for d in route_data if d['id'] == '1']
route_1_link = route_1[0]['links']['self']
route_1_data = requests.get('https://api-v3.mbta.com/routes/1')
route_1_data.json()

{'data': {'attributes': {'color': 'FFC72C',
   'description': 'Key Bus Route (Frequent Service)',
   'direction_names': ['Outbound', 'Inbound'],
   'long_name': '',
   'short_name': '1',
   'sort_order': 100,
   'text_color': '000000',
   'type': 3},
  'id': '1',
  'links': {'self': '/routes/1'},
  'type': 'route'},
 'jsonapi': {'version': '1.0'}}

In [7]:
vehicle_request = requests.get(mbta_api+'vehicles/').json()['data']

In [8]:
red_line_trains = [v for v in vehicle_request if v['relationships']['route']['data']['id'] == 'Red']
red_line_trains[0]

{'attributes': {'bearing': 330.0,
  'current_status': 'INCOMING_AT',
  'current_stop_sequence': 60,
  'direction_id': 1,
  'label': '1612',
  'last_updated': '2018-04-29T13:48:52-04:00',
  'latitude': 42.29161071777344,
  'longitude': -71.06536865234375,
  'speed': None},
 'id': 'R-54554657',
 'links': {'self': '/vehicles/R-54554657'},
 'relationships': {'route': {'data': {'id': 'Red', 'type': 'route'}},
  'stop': {'data': {'id': '70092', 'type': 'stop'}},
  'trip': {'data': {'id': 'ADDED-1524988080', 'type': 'trip'}}},
 'type': 'vehicle'}

In [9]:
red_line_train_coords = [ (train['attributes']['latitude'], train['attributes']['longitude']) for train in red_line_trains ]
rltc_array = np.asarray(red_line_train_coords)
rltc_array

array([[ 42.29161072, -71.06536865],
       [ 42.37960052, -71.12030029],
       [ 42.28868866, -71.06497955],
       [ 42.35591125, -71.06149292],
       [ 42.34273148, -71.05706787],
       [ 42.35266113, -71.05535889],
       [ 42.22515869, -71.00386047],
       [ 42.29592896, -71.04989624],
       [ 42.34064102, -71.05708313],
       [ 42.37636185, -71.11910248],
       [ 42.36278152, -71.0892868 ],
       [ 42.22504044, -71.00383759],
       [ 42.3961792 , -71.14054108]])

In [10]:
# plot the trains on a map
fig=gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(rltc_array))
fig

Figure(layout=FigureLayout(height='420px'))